In [1]:
from datasets import load_dataset, DatasetDict

In [2]:
raw_datasets = load_dataset("m-a-p/Code-Feedback")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'messages'],
        num_rows: 66383
    })
})

In [3]:
# SPLIT data in train/test

indices_1 = range(0,100)
indices_2 = range(101,201)
dataset_dict = {
    "train": raw_datasets["train"].select(indices_1),
    "test": raw_datasets["train"].select(indices_2)
}
raw_dataset = DatasetDict(dataset_dict)
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'messages'],
        num_rows: 100
    })
    test: Dataset({
        features: ['id', 'messages'],
        num_rows: 100
    })
})

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


tokenizer.pad_token_id = tokenizer.eos_token_id

#DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
#LLAMA_3_CHAT_TEMPLATE="{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"
LLAMA_3_CHAT_TEMPLATE = (
    "{% for message in messages %}"
        "{% if message['role'] == 'system' %}"
            "{{ message['content'] }}"
        "{% elif message['role'] == 'user' %}"
            "{{ '\n\nHuman: ' + message['content'] +  eos_token }}"
        "{% elif message['role'] == 'assistant' %}"
            "{{ '\n\nAssistant: '  + message['content'] +  eos_token  }}"
        "{% endif %}"
    "{% endfor %}"
    "{% if add_generation_prompt %}"
    "{{ '\n\nAssistant: ' }}"
    "{% endif %}"
)

tokenizer.chat_template = LLAMA_3_CHAT_TEMPLATE


In [16]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
 ]
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
print(tokenizer.decode(tokenized_chat[0]))

You are a friendly chatbot who always responds in the style of a pirate

Human: How many helicopters can a human eat in one sitting?<|end_of_text|>

Assistant: 
